In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.callbacks import TensorBoard
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

2025-01-26 08:46:42.069227: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-26 08:46:42.077925: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-26 08:46:42.088017: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-26 08:46:42.091064: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-26 08:46:42.098560: I tensorflow/core/platform/cpu_feature_guar

In [2]:
class Model(keras.Model):
    def __init__(self, dropout=0.0):
        super(Model, self).__init__()
        self.flatten = keras.layers.Flatten()
        self.dense1 = keras.layers.Dense(100, activation="relu")
        self.dense2 = keras.layers.Dense(100, activation="relu")
        self.dense3 = keras.layers.Dense(100, activation="relu")
        self.output_layer = keras.layers.Dense(10, activation="softmax")
        self.dropout = keras.layers.Dropout(dropout)
        self.callbacks = []

    def call(self, x):
        x = self.flatten(x)
        x = self.dropout(x)
        x = self.dense1(x)
        x = self.dropout(x)
        x = self.dense2(x)
        x = self.dropout(x)
        x = self.dense3(x)
        x = self.dropout(x)
        return self.output_layer(x)

    def train_loop(
        self, x_train, y_train, x_test, y_test, run_name, batch_size=32, epochs=20, loss="sparse_categorical_crossentropy"
    ):
        self.callbacks.append(TensorBoard(log_dir=f"./logs/{run_name}", histogram_freq=1))
        self.compile(
            optimizer="adam",
            loss=loss,
            metrics=["accuracy"],
        )
        self.fit(
            x_train,
            y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            callbacks=self.callbacks,
        )

In [3]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
model = Model()
model.train_loop(x_train, y_train, x_test, y_test, "no-reg", batch_size=512)

Epoch 1/20


I0000 00:00:1737861404.297376   99112 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-01-26 08:46:44.321479: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6092 - loss: 10.5992 - val_accuracy: 0.8711 - val_loss: 0.8313
Epoch 2/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8828 - loss: 0.6645 - val_accuracy: 0.8948 - val_loss: 0.5755
Epoch 3/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9163 - loss: 0.4042 - val_accuracy: 0.9159 - val_loss: 0.4460
Epoch 4/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9378 - loss: 0.2583 - val_accuracy: 0.9258 - val_loss: 0.3732
Epoch 5/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9518 - loss: 0.1902 - val_accuracy: 0.9320 - val_loss: 0.3389
Epoch 6/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9607 - loss: 0.1436 - val_accuracy: 0.9320 - val_loss: 0.3273
Epoch 7/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9645 - loss: 0.1284 - val_accuracy: 0.9387 - val_loss: 0.2965
Epoch 8/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9716 - loss: 0.0957 - val_accuracy: 0.9389 - val

# L1 Reg

In [5]:
model = Model()
model.dense1 = keras.layers.Dense(100, activation="relu", kernel_regularizer=keras.regularizers.l1(0.0001))
model.dense2 = keras.layers.Dense(100, activation="relu", kernel_regularizer=keras.regularizers.l1(0.0001))
model.dense3 = keras.layers.Dense(100, activation="relu", kernel_regularizer=keras.regularizers.l1(0.0001))
model.train_loop(x_train, y_train, x_test, y_test, "l1-reg", batch_size=512)

Epoch 1/20


/home/shusrith/projects/torch/lib/python3.12/site-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'model_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6922 - loss: 8.3017 - val_accuracy: 0.8825 - val_loss: 1.2432
Epoch 2/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8956 - loss: 1.0472 - val_accuracy: 0.9083 - val_loss: 0.9340
Epoch 3/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9263 - loss: 0.7712 - val_accuracy: 0.9237 - val_loss: 0.7953
Epoch 4/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9429 - loss: 0.6403 - val_accuracy: 0.9317 - val_loss: 0.7315
Epoch 5/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9562 - loss: 0.5523 - val_accuracy: 0.9333 - val_loss: 0.6904
Epoch 6/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9633 - loss: 0.5109 - val_accuracy: 0.9408 - val_loss: 0.6551
Epoch 7/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9707 - loss: 0.4644 - val_accuracy: 0.9436 - val_loss: 0.6386
Epoch 8/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9740 - loss: 0.4420 - val_accuracy: 0.9461 - val_

# L2 reg

In [6]:
model = Model()
model.dense1 = keras.layers.Dense(100, activation="relu", kernel_regularizer=keras.regularizers.l2(0.0001))
model.dense2 = keras.layers.Dense(100, activation="relu", kernel_regularizer=keras.regularizers.l2(0.0001))
model.dense3 = keras.layers.Dense(100, activation="relu", kernel_regularizer=keras.regularizers.l2(0.0001))
model.train_loop(x_train, y_train, x_test, y_test, "l2-reg", batch_size=512)

Epoch 1/20


/home/shusrith/projects/torch/lib/python3.12/site-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'model_2', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6151 - loss: 12.0233 - val_accuracy: 0.8664 - val_loss: 0.9684
Epoch 2/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8782 - loss: 0.7713 - val_accuracy: 0.8958 - val_loss: 0.6188
Epoch 3/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9146 - loss: 0.4427 - val_accuracy: 0.9111 - val_loss: 0.4898
Epoch 4/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9331 - loss: 0.3239 - val_accuracy: 0.9197 - val_loss: 0.4058
Epoch 5/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9444 - loss: 0.2456 - val_accuracy: 0.9241 - val_loss: 0.3667
Epoch 6/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9540 - loss: 0.2027 - val_accuracy: 0.9322 - val_loss: 0.3358
Epoch 7/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9623 - loss: 0.1675 - val_accuracy: 0.9353 - val_loss: 0.3312
Epoch 8/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9699 - loss: 0.1390 - val_accuracy: 0.9380 - val

# Data aug

In [7]:
data_augmemtation = keras.Sequential([
    keras.layers.RandomTranslation(0.1, 0.1),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.055),
])

xtrain_aug, xtest_aug = data_augmemtation(x_train.reshape((-1, 1, 28, 28))), data_augmemtation(x_test.reshape((-1, 1, 28, 28)))

In [8]:
model = Model()
model.train_loop(xtrain_aug, y_train, xtest_aug, y_test, "data-aug", batch_size=512)

Epoch 1/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4797 - loss: 14.0270 - val_accuracy: 0.7673 - val_loss: 1.0628
Epoch 2/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7935 - loss: 0.8883 - val_accuracy: 0.8439 - val_loss: 0.6208
Epoch 3/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8645 - loss: 0.5217 - val_accuracy: 0.8767 - val_loss: 0.4746
Epoch 4/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8867 - loss: 0.4043 - val_accuracy: 0.8938 - val_loss: 0.4042
Epoch 5/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9097 - loss: 0.3157 - val_accuracy: 0.8991 - val_loss: 0.3729
Epoch 6/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9226 - loss: 0.2630 - val_accuracy: 0.9114 - val_loss: 0.3291
Epoch 7/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9300 - loss: 0.2287 - val_accuracy: 0.9139 - val_loss: 0.3147
Epoch 8/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9403 - loss: 0.1976 - val_accuracy: 0

# Input noise

In [9]:
import numpy as np
noise = np.random.normal(0, 0.2, x_train.shape)
x_train_noise = x_train + noise
model = Model()
model.train_loop(x_train_noise, y_train, x_test, y_test, "input-noise", batch_size=512)

Epoch 1/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6473 - loss: 10.5509 - val_accuracy: 0.8744 - val_loss: 0.8232
Epoch 2/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8873 - loss: 0.6908 - val_accuracy: 0.9013 - val_loss: 0.5393
Epoch 3/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9207 - loss: 0.4062 - val_accuracy: 0.9189 - val_loss: 0.4332
Epoch 4/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9347 - loss: 0.2771 - val_accuracy: 0.9270 - val_loss: 0.3635
Epoch 5/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9472 - loss: 0.2105 - val_accuracy: 0.9281 - val_loss: 0.3429
Epoch 6/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9543 - loss: 0.1688 - val_accuracy: 0.9315 - val_loss: 0.3058
Epoch 7/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9630 - loss: 0.1374 - val_accuracy: 0.9424 - val_loss: 0.2817
Epoch 8/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9703 - loss: 0.1047 - val_accuracy: 0

# Output noise

In [10]:
import numpy as np
noise = abs(np.random.normal(0, 0.1, y_train.shape))
noise = np.expand_dims(noise, 1)
print(noise.shape)
y_train_one_hot = keras.utils.to_categorical(y_train, 10)
label = np.argmax(y_train_one_hot, axis=1)
y_train_noise = y_train_one_hot + (noise / 9)
y_train_noise[y_train_noise > 1] = 1 - noise[:, 0]
sum(y_train_noise[5]), y_train_noise[5]

(60000, 1)


(0.9999999999999997,
 array([4.07939014e-04, 4.07939014e-04, 9.96328549e-01, 4.07939014e-04,
        4.07939014e-04, 4.07939014e-04, 4.07939014e-04, 4.07939014e-04,
        4.07939014e-04, 4.07939014e-04]))

In [11]:
model = Model()
model.train_loop(x_train, y_train_noise, x_test, keras.utils.to_categorical(y_test, 10), "output-noise", batch_size=512, loss="categorical_crossentropy")

Epoch 1/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5503 - loss: 11.1313 - val_accuracy: 0.7568 - val_loss: 0.8347
Epoch 2/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7856 - loss: 1.2084 - val_accuracy: 0.8441 - val_loss: 0.5724
Epoch 3/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8590 - loss: 0.9403 - val_accuracy: 0.8777 - val_loss: 0.4614
Epoch 4/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8888 - loss: 0.8358 - val_accuracy: 0.8928 - val_loss: 0.4259
Epoch 5/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9109 - loss: 0.7699 - val_accuracy: 0.9051 - val_loss: 0.3818
Epoch 6/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9233 - loss: 0.7269 - val_accuracy: 0.9151 - val_loss: 0.3498
Epoch 7/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9340 - loss: 0.6950 - val_accuracy: 0.9190 - val_loss: 0.3351
Epoch 8/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9371 - loss: 0.6765 - val_accuracy: 0

# Dropout

In [12]:
model = Model(dropout=0.5)
model.train_loop(x_train, y_train, x_test, y_test, "dropout", batch_size=512)

Epoch 1/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6367 - loss: 10.5831 - val_accuracy: 0.8839 - val_loss: 0.8934
Epoch 2/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8896 - loss: 0.7304 - val_accuracy: 0.9047 - val_loss: 0.5514
Epoch 3/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9217 - loss: 0.3966 - val_accuracy: 0.9192 - val_loss: 0.4305
Epoch 4/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9379 - loss: 0.2653 - val_accuracy: 0.9250 - val_loss: 0.3774
Epoch 5/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9520 - loss: 0.1909 - val_accuracy: 0.9352 - val_loss: 0.3432
Epoch 6/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9593 - loss: 0.1589 - val_accuracy: 0.9374 - val_loss: 0.3289
Epoch 7/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9650 - loss: 0.1322 - val_accuracy: 0.9413 - val_loss: 0.2877
Epoch 8/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9741 - loss: 0.0917 - val_accuracy: 0

# Weight initialization

# zeroes

In [13]:
model = Model()
model.dense1 = keras.layers.Dense(100, activation="relu", kernel_initializer="zeros")
model.dense2 = keras.layers.Dense(100, activation="relu", kernel_initializer="zeros")
model.dense3 = keras.layers.Dense(100, activation="relu", kernel_initializer="zeros")
model.train_loop(x_train, y_train, x_test, y_test, "zeroes", batch_size=512)

Epoch 1/20


/home/shusrith/projects/torch/lib/python3.12/site-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'model_7', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1091 - loss: 2.3022 - val_accuracy: 0.1135 - val_loss: 2.3013
Epoch 2/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1110 - loss: 2.3013 - val_accuracy: 0.1135 - val_loss: 2.3011
Epoch 3/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1134 - loss: 2.3009 - val_accuracy: 0.1135 - val_loss: 2.3010
Epoch 4/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1130 - loss: 2.3010 - val_accuracy: 0.1135 - val_loss: 2.3010
Epoch 5/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1125 - loss: 2.3010 - val_accuracy: 0.1135 - val_loss: 2.3010
Epoch 6/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1129 - loss: 2.3012 - val_accuracy: 0.1135 - val_loss: 2.3010
Epoch 7/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1135 - loss: 2.3010 - val_accuracy: 0.1135 - val_loss: 2.3010
Epoch 8/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1129 - loss: 2.3013 - val_accuracy: 0.1135 - val_

# ones

In [14]:
model = Model()
model.dense1 = keras.layers.Dense(100, activation="relu", kernel_initializer="ones")
model.dense2 = keras.layers.Dense(100, activation="relu", kernel_initializer="ones")
model.dense3 = keras.layers.Dense(100, activation="relu", kernel_initializer="ones")
model.train_loop(x_train, y_train, x_test, y_test, "ones", batch_size=512)

Epoch 1/20


/home/shusrith/projects/torch/lib/python3.12/site-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'model_8', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0994 - loss: 174533360.0000 - val_accuracy: 0.1135 - val_loss: 17460198.0000
Epoch 2/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1001 - loss: 15861674.0000 - val_accuracy: 0.1135 - val_loss: 11518566.0000
Epoch 3/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0999 - loss: 13489444.0000 - val_accuracy: 0.0892 - val_loss: 26496816.0000
Epoch 4/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1010 - loss: 12520201.0000 - val_accuracy: 0.1028 - val_loss: 10307301.0000
Epoch 5/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0994 - loss: 10628133.0000 - val_accuracy: 0.0892 - val_loss: 16969206.0000
Epoch 6/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0989 - loss: 12659368.0000 - val_accuracy: 0.0892 - val_loss: 5267107.0000
Epoch 7/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0941 - loss: 7470621.5000 - val_accuracy: 0.1032 - val_loss: 9374368.0000
Epoch 8/20
118/118 

# random normal

In [15]:
model = Model()
model.dense1 = keras.layers.Dense(100, activation="relu", kernel_initializer="random_normal")
model.dense2 = keras.layers.Dense(100, activation="relu", kernel_initializer="random_normal")
model.dense3 = keras.layers.Dense(100, activation="relu", kernel_initializer="random_normal")
model.train_loop(x_train, y_train, x_test, y_test, "random-normal", batch_size=512)

Epoch 1/20


/home/shusrith/projects/torch/lib/python3.12/site-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'model_9', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6257 - loss: 3.9952 - val_accuracy: 0.9009 - val_loss: 0.3316
Epoch 2/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9161 - loss: 0.2818 - val_accuracy: 0.9285 - val_loss: 0.2361
Epoch 3/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9456 - loss: 0.1793 - val_accuracy: 0.9429 - val_loss: 0.1983
Epoch 4/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9602 - loss: 0.1288 - val_accuracy: 0.9494 - val_loss: 0.1767
Epoch 5/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9682 - loss: 0.1003 - val_accuracy: 0.9519 - val_loss: 0.1707
Epoch 6/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9748 - loss: 0.0777 - val_accuracy: 0.9567 - val_loss: 0.1568
Epoch 7/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9809 - loss: 0.0649 - val_accuracy: 0.9559 - val_loss: 0.1678
Epoch 8/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9842 - loss: 0.0535 - val_accuracy: 0.9595 - val_

# glorot normal - xavier

In [16]:
model = Model()
model.dense1 = keras.layers.Dense(100, activation="relu", kernel_initializer="glorot_normal")
model.dense2 = keras.layers.Dense(100, activation="relu", kernel_initializer="glorot_normal")
model.dense3 = keras.layers.Dense(100, activation="relu", kernel_initializer="glorot_normal")
model.train_loop(x_train, y_train, x_test, y_test, "glorot-normal", batch_size=512)

Epoch 1/20


/home/shusrith/projects/torch/lib/python3.12/site-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'model_10', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6717 - loss: 8.3134 - val_accuracy: 0.8830 - val_loss: 0.8250
Epoch 2/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8943 - loss: 0.6935 - val_accuracy: 0.9090 - val_loss: 0.5252
Epoch 3/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9230 - loss: 0.3906 - val_accuracy: 0.9190 - val_loss: 0.4516
Epoch 4/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9391 - loss: 0.2683 - val_accuracy: 0.9294 - val_loss: 0.3733
Epoch 5/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9521 - loss: 0.1895 - val_accuracy: 0.9328 - val_loss: 0.3517
Epoch 6/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9611 - loss: 0.1477 - val_accuracy: 0.9392 - val_loss: 0.3247
Epoch 7/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9666 - loss: 0.1150 - val_accuracy: 0.9414 - val_loss: 0.3031
Epoch 8/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9736 - loss: 0.0880 - val_accuracy: 0.9436 - val_

# he normal

In [17]:
model = Model()
model.dense1 = keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal")
model.dense2 = keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal")
model.dense3 = keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal")
model.train_loop(x_train, y_train, x_test, y_test, "he-normal", batch_size=512)

Epoch 1/20


/home/shusrith/projects/torch/lib/python3.12/site-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'model_11', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5944 - loss: 27.4407 - val_accuracy: 0.8680 - val_loss: 1.7328
Epoch 2/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8815 - loss: 1.4060 - val_accuracy: 0.8962 - val_loss: 1.0577
Epoch 3/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9072 - loss: 0.7966 - val_accuracy: 0.9059 - val_loss: 0.8123
Epoch 4/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9237 - loss: 0.5319 - val_accuracy: 0.9147 - val_loss: 0.6737
Epoch 5/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9363 - loss: 0.3892 - val_accuracy: 0.9202 - val_loss: 0.6143
Epoch 6/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9472 - loss: 0.2950 - val_accuracy: 0.9233 - val_loss: 0.5598
Epoch 7/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9525 - loss: 0.2464 - val_accuracy: 0.9276 - val_loss: 0.5095
Epoch 8/20
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9585 - loss: 0.1962 - val_accuracy: 0.9315 - val